# lightgbm期货交易

# 数据导入

In [3]:
import pandas as pd
import talib
import numpy as np
from numpy.linalg import eig
import math
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# from xgboost import plot_importance
from matplotlib import pyplot as plt
import datetime
# import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

ModuleNotFoundError: No module named 'lightgbm'

In [4]:
data = pd.read_csv('D://桌面/future_data.csv')
data.head(),data.shape

FileNotFoundError: [Errno 2] No such file or directory: 'D:/桌面/future_data.csv'

In [3]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data = data.iloc[530000:,:]
data.head()

,date,open,close,high,low,volume,open_interest
530000,2017-09-05 10:06:00,2727.0,2728.0,2728.0,2726.0,1270.0,1871890.0
530001,2017-09-05 10:07:00,2727.0,2727.0,2728.0,2727.0,676.0,1871784.0
530002,2017-09-05 10:08:00,2728.0,2728.0,2728.0,2727.0,202.0,1871836.0
530003,2017-09-05 10:09:00,2728.0,2728.0,2728.0,2727.0,222.0,1871758.0
530004,2017-09-05 10:10:00,2728.0,2727.0,2728.0,2727.0,2480.0,1872288.0


In [4]:
data.shape

(280150, 7)

# 数据预处理

In [5]:
p_data = data[['date','close','high','low','volume','open_interest']].reset_index(drop = True)
p_data['d_high'] = p_data['high']
p_data['d_close'] = p_data['close']
p_data['d_low'] = p_data['low']
p_data['d_insterest'] = p_data['open_interest']
p_data['Y'] = p_data['close']
p_data.head()

,date,close,high,low,volume,open_interest,d_high,d_close,d_low,d_insterest,Y
0,2017-09-05 10:06:00,2728.0,2728.0,2726.0,1270.0,1871890.0,2728.0,2728.0,2726.0,1871890.0,2728.0
1,2017-09-05 10:07:00,2727.0,2728.0,2727.0,676.0,1871784.0,2728.0,2727.0,2727.0,1871784.0,2727.0
2,2017-09-05 10:08:00,2728.0,2728.0,2727.0,202.0,1871836.0,2728.0,2728.0,2727.0,1871836.0,2728.0
3,2017-09-05 10:09:00,2728.0,2728.0,2727.0,222.0,1871758.0,2728.0,2728.0,2727.0,1871758.0,2728.0
4,2017-09-05 10:10:00,2727.0,2728.0,2727.0,2480.0,1872288.0,2728.0,2727.0,2727.0,1872288.0,2727.0


In [6]:
def transfrom_data(p_data,peroid,p_peroid):
    # 处理数据格式
    p_data['d_high'][1:p_data.shape[0] - peroid] = p_data['high'][1:p_data.shape[0] - peroid].reset_index(drop = True) - p_data['close'][0:p_data.shape[0] - peroid - 1].reset_index(drop = True)
    p_data['d_low'][1:p_data.shape[0] - peroid] = p_data['d_low'][1:p_data.shape[0] - peroid].reset_index(drop = True) - p_data['close'][0:p_data.shape[0] - peroid - 1].reset_index(drop = True)
    p_data['d_close'][1:p_data.shape[0] - peroid] = p_data['d_close'][1:p_data.shape[0] - peroid].reset_index(drop = True) - p_data['close'][0:p_data.shape[0] - peroid - 1].reset_index(drop = True)
    p_data['d_insterest'][1:p_data.shape[0] - peroid] = p_data['d_insterest'][1:p_data.shape[0] - peroid].reset_index(drop = True) - p_data['open_interest'][0:p_data.shape[0] - peroid - 1].reset_index(drop = True)

    #     p_data['Y'][1:p_data.shape[0] - peroid] = (p_data['close'][1 + p_peroid:p_data.shape[0] - peroid + p_peroid].reset_index(drop = True) - p_data['close'][1:p_data.shape[0] - peroid].reset_index(drop = True)) / p_data['close'][1:p_data.shape[0] - peroid].reset_index(drop = True)
    p_data['Y'][1:p_data.shape[0] - peroid] = p_data['close'][1 + p_peroid:p_data.shape[0] - peroid + p_peroid].reset_index(drop = True) - p_data['close'][1:p_data.shape[0] - peroid].reset_index(drop = True)
    
    o_data = p_data.iloc[1:p_data.shape[0] - peroid,[0,4,6,7,8,9,10]].reset_index(drop = True)
    
    X = o_data.values[:,1:-1]
    Time = o_data.values[:,0]
    Y = o_data.values[:,-1]
    data_x = np.zeros((X.shape[0],X.shape[1] * peroid))
    
    # 转换时间周期
    for i in range(peroid):    
        data_x[peroid:,i * 5 : i * 5 + 5] = X[peroid - i:X.shape[0]-i,:]
    data_X = data_x[peroid:,:]
    data_Time = Time[peroid:]
    data_Y = Y[peroid:]
    
    # 时间筛选
    data_Time = pd.DataFrame(data_Time)
    d_list = list(data_Time[data_Time[0].str.contains(' 09:')].index.values) + list(data_Time[data_Time[0].str.contains(' 21:')].index.values)
    time_list = []
    k = 0
    for i in range(data_Time.shape[0]):
        if k == len(d_list) or i != d_list[k]:
            time_list.append(i)
        else:
            k += 1
            
    data_X = pd.DataFrame(data_X).iloc[time_list,:]
    data_X = data_X.reset_index(drop = True)
    data_Y = pd.DataFrame(data_Y).iloc[time_list,:]
    data_Y = data_Y.reset_index(drop = True)
    data_Time = pd.DataFrame(data_Time).iloc[time_list,:].reset_index(drop = True)
    
    mean = data_X.mean()
    std = data_X.std()
    data_X = preprocessing.scale(data_X)
    data_X = pd.DataFrame(data_X)
#     yuezhi = 3 / 3000
    yuezhi = 3
    data_list = list(data_Y[abs(data_Y[0]) > yuezhi].index.values)
    d_X = data_X.iloc[data_list,:].reset_index(drop = True)
    d_Y = data_Y.iloc[data_list,:].reset_index(drop = True)
    d_T = data_Time.iloc[data_list,:].reset_index(drop = True)
    
    return [d_X,d_Y,d_T,mean,std]


In [70]:
peroid = 30
p_peroid = 5
d_X,d_Y,d_T,mean,std = transfrom_data(p_data,peroid,p_peroid)

<ipython-input-69-686cdef7fc54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_data['d_high'][1:p_data.shape[0] - peroid] = p_data['high'][1:p_data.shape[0] - peroid].reset_index(drop = True) - p_data['close'][0:p_data.shape[0] - peroid - 1].reset_index(drop = True)
<ipython-input-69-686cdef7fc54>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_data['d_low'][1:p_data.shape[0] - peroid] = p_data['d_low'][1:p_data.shape[0] - peroid].reset_index(drop = True) - p_data['close'][0:p_data.shape[0] - peroid - 1].reset_index(drop = True)
<ipython-input-69-686cdef7fc54>:5: SettingWithCopyWarning: 
A value is trying to be 

In [71]:
d_Y

,0
0,-4.0
1,5.0
2,4.0
3,5.0
4,4.0
...,...
39910,8.0
39911,4.0
39912,4.0
39913,5.0


In [67]:
# 寻找不同交易周期和数据周期下的最优解
peroid_list = [5,10,15,20,25,30]
p_peroid_list = [i + 1 for i in range(10)]
train_acc = []
verify_acc = []
test_acc = []

In [72]:
tt = d_X.shape[0] // 5 * 4

back_test_data_x = d_X.iloc[tt:,:].copy() 
back_test_time = d_T.iloc[tt:,:].copy() 
back_test_data_y = d_Y.iloc[tt:,:].copy() 

# 训练测试集
X = d_X.iloc[:tt,:].copy()
Y = d_Y.iloc[:tt,:].copy()

yuezhi = 0
Y[Y[0] > yuezhi] = 1
Y[Y[0] < -yuezhi] = 0

X_train, X_test, y_train, y_test = train_test_split(np.array(X),list(np.array(Y)[:,0]), test_size=0.3)
# X_train, X_test, y_train, y_test = train_test_split(np.array(X),np.array(Y), test_size=0.3)


# 建立模型

In [79]:
lgb_train = lgb.Dataset(X_train, y_train) # 将数据保存到LightGBM二进制文件将使加载更快
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)  # 创建验证数据
 
# 将参数写成字典下形式
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'binary', # 目标函数
    'metric': {'l2', 'auc'},  # 评估函数
    'num_leaves': 31,   # 叶子节点数
    'learning_rate': 0.01,  # 学习速率
    'feature_fraction': 0.9, # 建树的特征选择比例
    'bagging_fraction': 0.8, # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
 
gbm = lgb.train(params,lgb_train,num_boost_round=100,valid_sets=lgb_eval) 


[LightGBM] [Info] Number of positive: 11298, number of negative: 11054
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013060 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31346
[LightGBM] [Info] Number of data points in the train set: 22352, number of used features: 150
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.505458 -> initscore=0.021833
[LightGBM] [Info] Start training from score 0.021833
[1]	valid_0's auc: 0.522125	valid_0's l2: 0.249871
[2]	valid_0's auc: 0.525838	valid_0's l2: 0.24984
[3]	valid_0's auc: 0.523477	valid_0's l2: 0.249818
[4]	valid_0's auc: 0.524575	valid_0's l2: 0.249787
[5]	valid_0's auc: 0.524346	valid_0's l2: 0.249764
[6]	valid_0's auc: 0.528506	valid_0's l2: 0.249728
[7]	valid_0's auc: 0.530039	valid_0's l2: 0.249694
[8]	valid_0's auc: 0.533021	valid_0's l2: 0.249649
[9]	valid_0's auc: 0.534523	valid_0's l2: 0.249617
[10]	valid_0's auc: 0.538525	valid_0's l2: 0.

In [80]:
y_pred1=gbm.predict(X_train, num_iteration=gbm.best_iteration)
# # model.save_model('testXGboostClass.model')  # 保存训练模型
# yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro
pred1 = [round(value) for value in y_pred1]
true1 = [round(value) for value in y_train]
# evaluate predictions
accuracy = accuracy_score(true1,pred1 )

print("训练集Accuracy: %.2f%%" % (accuracy * 100.0))

y_pred=gbm.predict(X_test, num_iteration=gbm.best_iteration)
pred = [round(value) for value in y_pred]
true = [round(value) for value in y_test]
# evaluate predictions
accuracy = accuracy_score(true,pred )

print("验证集Accuracy: %.2f%%" % (accuracy * 100.0))
back_test_data_y[back_test_data_y[0] >= 0] = 1
back_test_data_y[back_test_data_y[0] < 0] = 0
y_pred1=gbm.predict(back_test_data_x, num_iteration=gbm.best_iteration)
pred1 = [round(value) for value in y_pred1]
true1 = [round(value) for value in np.array(back_test_data_y)[:,0]]
# evaluate predictions
accuracy = accuracy_score(true1,pred1 )
print("测试集Accuracy: %.2f%%" % (accuracy * 100.0))

训练集Accuracy: 64.08%
验证集Accuracy: 53.94%
测试集Accuracy: 52.06%


In [56]:
back_test_data_y.describe()

,0
count,7983
unique,1
top,1
freq,7983


In [57]:
back_test_data_y[0]

31932    1
31933    1
31934    1
31935    1
31936    1
        ..
39910    1
39911    1
39912    1
39913    1
39914    1
Name: 0, Length: 7983, dtype: object

In [ ]:
accuracy_score()